In [1]:
import numpy as np
from datetime import datetime
import smtplib
import time
from selenium import webdriver
import os

In [2]:
#For Prediction
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, svm

In [3]:
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data

In [4]:
def getStocks(n):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    url="https://finance.yahoo.com/screener/predefined/aggressive_small_caps?offset=0&count=202"
    driver.get(url)
    stock_list = []
    n += 1
    for i in range(1, n):
        ticker = driver.find_element_by_xpath('//*[@id="scr-res-table"]/div[1]/table/tbody/tr['+str(i)+']/td[1]')
        stock_list.append(ticker.text)
    driver.quit()
    
    number = 0
    for i in stock_list:
        print("Number: "+str(number))
        try:
            predictData(i, 5)
        except:
            print("Stock: " + i + " was not predicted")
        number += 1
    

In [5]:
def predictData(stock, days):
    print(stock)

    start = datetime(2017, 1, 1)
    end = datetime.now()

    #Outputting the Historical data into a .csv for later use
    df = get_historical_data(stock,start=start, end=end,output_format='pandas')
    if os.path.exists('./Exports'):
        csv_name = ('Exports/'+ stock +'_Export.csv')
    else:
        os.mkdir("Exports")
        csv_name = ('Exports/'+ stock +'_Export.csv')
    df.to_csv(csv_name)
    df['prediction'] = df['close'].shift(-1)
    df.dropna(inplace=True)

    forecast_time = int(days)

    #Predicting the Stock price in the future
    X = np.array(df.drop(['prediction'], 1))
    Y = np.array(df['prediction'])
    X = preprocessing.scale(X)
    X_prediction = X[-forecast_time:]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

    #Performing the Regression on the training data
    clf = LinearRegression()
    clf.fit(X_train, Y_train)
    prediction = (clf.predict(X_prediction))

    last_row = df.tail(1)
    print(last_row['close'])

In [6]:
if __name__ == '__main__':
   getStocks(5)

Number: 0
AUY
date
2019-04-30    2.19
Name: close, dtype: float64
Number: 1
AKS
date
2019-04-30    2.42
Name: close, dtype: float64
Number: 2
DNR
date
2019-04-30    2.23
Name: close, dtype: float64
Number: 3
NE
date
2019-04-30    2.63
Name: close, dtype: float64
Number: 4
ESV
date
2019-04-30    13.97
Name: close, dtype: float64
